# Compile a Keras model with Sagemaker Neo for inference on Greengrass

Make sure you use the conda_tensorflow2_p36 kernel in SageMaker for this notebook!

For our sample image classification task we will use a pretrained [MobileNetV2](https://arxiv.org/abs/1801.04381) model from the [Keras applications](https://github.com/keras-team/keras-applications) module.
For a real world use case we would [fine-tune](https://www.tensorflow.org/tutorials/images/transfer_learning) the pretrained model based custom images and labels for our business problem. However as we are mostly focused on the deployment aspect, we will skip this step here!

## Downloading and saving the model 

As a first step we will download the model and save it in Keras in [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) format. Make sure to specify a bucket where to upload the model in the next cell!

In [1]:
#BUCKET = '<your_bucket_name>''
BUCKET = 'ml-gg-deployment-sample'

In [2]:
import boto3
import tensorflow as tf

IMG_SIZE = 224
IMAGE_SHAPE = (IMG_SIZE,IMG_SIZE,3)

# start clean in case of rerunning this cell
tf.keras.backend.clear_session()

classifier = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=True,
                                               weights='imagenet')
classifier.save("model.h5",save_format='h5')
!tar -zcvf model.tar.gz model.h5

model.h5


Upload compressed model to S3

In [3]:
model_key = 'models/mobilenet-keras/model.tar.gz'
boto3.resource('s3').Bucket(BUCKET).upload_file('model.tar.gz', model_key)

# Compile model the model using Sagemaker Neo

In order to optimize the model for inference on Greegrass we will use [Sagemaker Neo](https://docs.aws.amazon.com/sagemaker/latest/dg/neo.html) to compile the model into the format required by the SageMaker Neo runtime.

In [4]:
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
sm_client = boto3.client('sagemaker')


Now we will kick off the SageMaker Neo compilation job. Check here for [supported frameworks](https://aws.amazon.com/releasenotes/sagemaker-neo-supported-frameworks-and-operators/) and [hardware platforms](https://aws.amazon.com/releasenotes/sagemaker-neo-supported-target-platforms-and-compiler-options/).

In [5]:
# create job name with timestamp
compilation_job_name = name_from_base('tf-mobilenetv2-neo')
data_shape = '{"input_1":[1,3,224,224]}'
target_device = 'ml_c5'
framework = 'KERAS'
framework_version = '2.3.1'
compiled_model_path = 's3://{}/models/mobilenet-neo/{}/'.format(BUCKET, compilation_job_name)
model_path = 's3://{}/{}'.format(BUCKET, model_key)

response = sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': model_path,
        'DataInputConfig': data_shape,
        'Framework': framework
    },
    OutputConfig={
        'S3OutputLocation': compiled_model_path,
        'TargetDevice': target_device
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 300
    }
)
print(response)

# Poll every 30 sec
while True:
    response = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

# Extract compiled model artifact
compiled_model_path = response['ModelArtifacts']['S3ModelArtifacts']

{'CompilationJobArn': 'arn:aws:sagemaker:eu-west-1:828266890613:compilation-job/tf-mobilenetv2-neo-2020-08-26-13-49-32-828', 'ResponseMetadata': {'RequestId': '67dd244d-a9a6-4698-bede-c6b586be6490', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '67dd244d-a9a6-4698-bede-c6b586be6490', 'content-type': 'application/x-amz-json-1.1', 'content-length': '123', 'date': 'Wed, 26 Aug 2020 13:49:32 GMT'}, 'RetryAttempts': 0}}
Compiling ...
Compiling ...
Compiling ...
Done!


Copy to static location in S3 bucket

In [6]:
!aws s3 cp $compiled_model_path s3://$BUCKET/ml_resources/neo/model.tar.gz

copy: s3://ml-gg-deployment-sample/models/mobilenet-neo/tf-mobilenetv2-neo-2020-08-26-13-49-32-828/model-ml_c5.tar.gz to s3://ml-gg-deployment-sample/ml_resources/neo/model.tar.gz
